In [2]:
from pyspark.sql import SparkSession


In [3]:
spark= SparkSession.builder.appName("q1").getOrCreate()

24/11/02 19:45:51 WARN Utils: Your hostname, Anants-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.68.3.53 instead (on interface en0)
24/11/02 19:45:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/02 19:45:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


1) Write a PySpark program to square set of integers.

In [5]:
numbers=[1,2,3,4,5]

In [6]:
rdd=spark.sparkContext.parallelize(numbers)
squared_rdd = rdd.map(lambda x: x ** 2)
squared_numbers = squared_rdd.collect()
print("RDD Approach:", squared_numbers)

RDD Approach: [1, 4, 9, 16, 25]


In [7]:
from pyspark.sql.functions import col

# Create a DataFrame from the list of integers
df = spark.createDataFrame([(n,) for n in numbers], ["number"])

squared_df = df.withColumn("squared", col("number") ** 2)
print("DataFrame Approach:")
# Show the results
squared_df.show()


DataFrame Approach:
+------+-------+
|number|squared|
+------+-------+
|     1|    1.0|
|     2|    4.0|
|     3|    9.0|
|     4|   16.0|
|     5|   25.0|
+------+-------+



2) Write a PySpark program to find the maximum of given set of numbers.

In [9]:
max_number = rdd.max()
print("RDD Approach - Max Number:", max_number)

from pyspark.sql.functions import max
max_number_df = df.agg(max("number").alias("max_number"))
max_number_df.show()

RDD Approach - Max Number: 5
+----------+
|max_number|
+----------+
|         5|
+----------+



3) Write a PySpark program to find average of N numbers.

In [11]:
average = rdd.mean()
print("RDD Approach - Average:", average)

from pyspark.sql.functions import avg

average_df = df.agg(avg("number").alias("average"))
average_df.show()


RDD Approach - Average: 3.0
+-------+
|average|
+-------+
|    3.0|
+-------+



In [12]:
csv_file_path = "business_operations.csv"
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(csv_file_path)
df.show()

+--------------------+--------------------+-----+--------------+---------+-----+
|         description|            industry|level|          size|line_code|value|
+--------------------+--------------------+-----+--------------+---------+-----+
|Type of outstandi...|               total|    0| 619 employees|    D0201|13215|
|Type of outstandi...|               total|    0|          2343|    D0201| 3405|
|Type of outstandi...|               total|    0|5099 employees|    D0201|  978|
|Type of outstandi...|               total|    0|100+ employees|    D0201|  687|
|Type of outstandi...|Agriculture, fore...|    1|         total|    D0201| 1974|
|Type of outstandi...|         Agriculture|    2|         total|    D0201| 1371|
|Type of outstandi...|  Commercial fishing|    2|         total|    D0201|   24|
|Type of outstandi...|  Forestry & logging|    2|         total|    D0201|  117|
|Type of outstandi...|Agriculture, fore...|    2|         total|    D0201|  465|
|Type of outstandi...|      

In [13]:
rdd = spark.sparkContext.textFile(csv_file_path)

header = rdd.first() 
data_rdd = rdd.filter(lambda row: row != header).map(lambda row: row.split(","))

for row in data_rdd.collect():
    print(row)

['Type of outstanding debt: bank overdrafts', 'total', '0', '619 employees', 'D0201', '13215']
['Type of outstanding debt: bank overdrafts', 'total', '0', '2343', 'D0201', '3405']
['Type of outstanding debt: bank overdrafts', 'total', '0', '5099 employees', 'D0201', '978']
['Type of outstanding debt: bank overdrafts', 'total', '0', '100+ employees', 'D0201', '687']
['Type of outstanding debt: bank overdrafts', '"Agriculture', ' forestry', ' & fishing"', '1', 'total', 'D0201', '1974']
['Type of outstanding debt: bank overdrafts', 'Agriculture', '2', 'total', 'D0201', '1371']
['Type of outstanding debt: bank overdrafts', 'Commercial fishing', '2', 'total', 'D0201', '24']
['Type of outstanding debt: bank overdrafts', 'Forestry & logging', '2', 'total', 'D0201', '117']
['Type of outstanding debt: bank overdrafts', '"Agriculture', ' forestry', ' & fishing support services"', '2', 'total', 'D0201', '465']
['Type of outstanding debt: bank overdrafts', 'Mining', '1', 'total', 'D0201', '30']
['

In [14]:
df.summary().show()

24/11/02 19:45:57 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+--------------------+--------------------+------------------+--------------+---------+-----------------+
|summary|         description|            industry|             level|          size|line_code|            value|
+-------+--------------------+--------------------+------------------+--------------+---------+-----------------+
|  count|                6627|                6627|              6627|          6627|     6627|             6627|
|   mean|                NULL|                NULL|1.4042553191489362|        2343.0|     NULL|421.9379809868719|
| stddev|                NULL|                NULL|0.6735483434489863|          NULL|     NULL|2071.697633085584|
|    min|Changes in credit...|Accommodation & f...|                 0|100+ employees|    D0201|                0|
|    25%|                NULL|                NULL|                 1|        2343.0|     NULL|                3|
|    50%|                NULL|                NULL|                 2|        2343.0|   

In [15]:
df.describe().show()

+-------+--------------------+--------------------+------------------+--------------+---------+-----------------+
|summary|         description|            industry|             level|          size|line_code|            value|
+-------+--------------------+--------------------+------------------+--------------+---------+-----------------+
|  count|                6627|                6627|              6627|          6627|     6627|             6627|
|   mean|                NULL|                NULL|1.4042553191489362|        2343.0|     NULL|421.9379809868719|
| stddev|                NULL|                NULL|0.6735483434489863|          NULL|     NULL|2071.697633085584|
|    min|Changes in credit...|Accommodation & f...|                 0|100+ employees|    D0201|                0|
|    max|Who provided equi...|               total|                 2|         total| D2100.04|            42840|
+-------+--------------------+--------------------+------------------+--------------+---

In [31]:
spark.stop()